In [6]:
!pip install transformers datasets scikit-learn torch
# emotions tested: Happiness, Sadness, Fear, Surprise, Neutral, Anger, Disgust

# original/prev dataset (no disgust or neutral): https://www.kaggle.com/datasets/parulpandey/emotion-dataset?resource=download
# dataset with 6 emotions (no neutral): https://www.kaggle.com/code/shainy/twitter-emotion-analysis/input
# dataset for neutral emotions: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis?select=twitter_training.csv

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\student\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [9]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl (390.3 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import zipfile
import os

# Define the path to the zip file
zip_path = './data.csv.zip'

# Define the directory where the files will be extracted
extract_path = './twitter_emotions/'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

zip_path = './twitter_training.csv.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the extracted files and directories
print("Extracted files and directories:")
print(os.listdir(extract_path))

Extracted files and directories:
['data.csv', 'twitter_training.csv']


In [3]:
import pandas as pd

# Paths to the extracted files
data_path = './twitter_emotions/data.csv'
sentiment_path = './twitter_emotions/twitter_training.csv'

# Load the CSV files into pandas DataFrames
data = pd.read_csv(data_path)
neutrals = pd.read_csv(sentiment_path)
data.drop('Search key', axis = 1, inplace=True)
data.drop('Sl no', axis = 1, inplace=True)

neutrals = neutrals.iloc[:, -2:]
neutrals.columns = ["Feeling", "Tweets"]
neutral_df = neutrals[neutrals['Feeling'] == 'Neutral']
combined_df = pd.concat([data, neutral_df])
combined_df['Tweets'] = combined_df['Tweets'].str.replace(r'#\d+: ', '', regex=True)
print(combined_df.head())



                                              Tweets Feeling
0   @fe ed "RT @MirayaDizon1: Time is ticking fas...   happy
1   @蓮花 &はすか ed "RT @ninjaryugo: ＃コナモンの日 だそうで、コナモ...   happy
2   @Ris ♡ ed "Happy birthday to one smokin hot m...   happy
3   @월월 [씍쯴사랑로봇] jwinnie is the best, cheer up jw...   happy
4   @Madhurima wth u vc♥ ed "Good morning dear❤ @...   happy


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

combined_df = combined_df.fillna('')  # Replace NaN values with an empty string
combined_df = combined_df.astype(str)  # Convert all elements to strings
X_train, X_test, y_train, y_test = train_test_split(combined_df['Tweets'], combined_df['Feeling'], test_size=0.2, random_state=42)


# Tokenization and Padding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np


# Pad sequences to the same length
max_length = max(max(len(seq) for seq in X_train_seq), 100)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

# Encode labels as integers
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert labels to one-hot encoding for categorical crossentropy
num_classes = len(np.unique(y_train_encoded))
y_train_one_hot = tf.keras.utils.to_categorical(y_train_encoded, num_classes=num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test_encoded, num_classes=num_classes)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional

# Model Architecture
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train_padded, y_train_one_hot,
    epochs=10,
    batch_size=32
)

with open('./nlp-model', 'wb') as f:
    pickle.dump(model, f)

with open('./label-encoder', 'wb') as f:
    pickle.dump(label_encoder, f)

with open('./tokenizer', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('./max-length', 'wb') as f:
    pickle.dump(max_length, f)

print("Max length:", max_length)
    
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test_one_hot)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/10


C:\Users\student\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


709/709 ━━━━━━━━━━━━━━━━━━━━ 113s 153ms/step - accuracy: 0.7732 - loss: 0.6838
Epoch 2/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 112s 157ms/step - accuracy: 0.9311 - loss: 0.2123
Epoch 3/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 114s 160ms/step - accuracy: 0.9659 - loss: 0.1122
Epoch 4/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 119s 167ms/step - accuracy: 0.9798 - loss: 0.0710
Epoch 5/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 118s 166ms/step - accuracy: 0.9870 - loss: 0.0443
Epoch 6/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 177s 250ms/step - accuracy: 0.9901 - loss: 0.0322
Epoch 7/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 118s 166ms/step - accuracy: 0.9908 - loss: 0.0292
Epoch 8/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 111s 157ms/step - accuracy: 0.9932 - loss: 0.0222
Epoch 9/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 118s 167ms/step - accuracy: 0.9938 - loss: 0.0176
Epoch 10/10
709/709 ━━━━━━━━━━━━━━━━━━━━ 125s 176ms/step - accuracy: 0.9943 - loss: 0.0173
Max length: 151
178/178 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9662 - loss: 0.1727
Test Loss: 0.18715

In [12]:
def predict_sentiment(sentence):
    # Convert the sentence to a sequence
    sequence = tokenizer.texts_to_sequences([sentence])
    
    # Pad the sequence to match the training data length
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    
    # Make prediction
    prediction = model.predict(padded_sequence)
    #print("Prediction:", prediction)
    # Get the class with the highest probability
    predicted_class = np.argmax(prediction)
    
    # Convert back to original label
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    
    return predicted_label, prediction[0]

# # Example usage
sentence = "I feel so joyful! Isn't this amazing?"
label, probabilities = predict_sentiment(sentence)
print(f"Predicted Sentiment: {label}")
print("Probabilities:", probabilities)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
Predicted Sentiment: Neutral
Probabilities: [9.9926859e-01 1.2669040e-05 7.5875946e-08 2.1452071e-07 6.7376986e-04
 2.8285509e-05 1.6322179e-05]


In [8]:
from sklearn.metrics import classification_report
import numpy as np
y_pred = model.predict(X_test_padded)
y_pred_labels = np.argmax(y_pred, axis=1)
emotion_labels = label_encoder.classes_

report = classification_report(y_test_encoded, y_pred_labels, target_names=emotion_labels)
print(report)

178/178 ━━━━━━━━━━━━━━━━━━━━ 27s 153ms/step
              precision    recall  f1-score   support

     Neutral       1.00      1.00      1.00      3682
       angry       0.85      0.89      0.87       280
     disgust       0.95      0.89      0.92       131
        fear       0.90      0.77      0.83       169
       happy       0.91      0.89      0.90       754
         sad       0.85      0.91      0.88       554
    surprise       0.95      0.84      0.89        97

    accuracy                           0.96      5667
   macro avg       0.92      0.88      0.90      5667
weighted avg       0.96      0.96      0.96      5667

